# Google Play Reviews Crawling & Analysis
## 예 ) Retrica App(사진어플)
 - 총 평점과 인원들을 조회하기 위해 첫페이지 크롤링
 - 리뷰 : ajax 통신을 통한 크롤링. 

In [3]:
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import requests
import json
import requests
import re
from bs4 import BeautifulSoup

In [2]:
%matplotlib inline

## 1.  전체 평점 및 투표인원
 - 첫 페이지 크롤링

In [4]:
url = 'https://play.google.com/store/apps/details?id=com.venticake.retrica&hl=ko#details-reviews'
headers = {'User-Agent': 'Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)'}
res = requests.get(url, headers=headers)

In [5]:
res

<Response [200]>

In [6]:
bs = BeautifulSoup(res.text, 'html.parser')
div_list = bs.find_all('div',class_="details-section-body expandable")
div_root_list = bs.find_all('div',class_="details-wrapper apps")

- 각 리뷰 스코오별 투표 인원 가지고 오기
- 1~5점 각 인원, 평균, 전체 인원

In [7]:
for div_list in div_root_list:
    rating = div_list.find_all('div',class_="rating-box")
    if len(rating)>0:
        total_num = rating[0].find_all('span', class_="reviews-num")[0].text
        avg = rating[0].find_all("div",class_="score")[0].text
        score_detail = rating[0].find_all("div",class_="rating-histogram")[0].find_all('div',class_="rating-bar-container")
        total_number = {}
        for score in score_detail:
            label = score.find('span',class_="bar-label").text.strip()
            label_number = score.find('span',class_="bar-number").text
            total_number[label] = label_number
        total_number['avg'] = avg
        total_number['total'] = total_num
        
total_series = pd.Series(total_number)
total_series

1          407,171
2          224,005
3          469,633
4          843,977
5        4,021,934
avg            4.3
total    5,966,720
dtype: object

## 2. 리뷰 전체 가지고 오기.
 - 필요 데이터 
  - id : com.venticake.retrica
  - url : https://play.google.com/store/getreviews?authuser=0
  - token : VgLHly2_FzRO3_28IkocTRfdZ1Q:1505134935641

![ajax](./img/2.png)

In [8]:
url = "https://play.google.com/store/getreviews?authuser=0"
id = 'com.venticake.retrica'
token = 'VgLHly2_FzRO3_28IkocTRfdZ1Q:1505134935641'

In [35]:
# Review가지고오기.
def GetReviews(url,id,token,pages):  
    param = {'reviewType': '0', 'pageNum': '10000', 'id':'','reviewSortOrder':'4','xhr':'1','token':'','hl':'ko'}
    param['id'] = id
    param['token'] = token
    
    review_date_all = []
    review_star_all = []
    review_user_all = []
    review_title_all = []
    review_body_all = []
    for i in range(1,pages):
        param['pageNum'] = i
        res = requests.post(url, data=param)
        try:
            if i % 20 == 0:
                print(" line : {line}, code : {code}".format(line=i,code=res.status_code))

            if res.status_code == 400 or i == pages-1:
                review_df = pd.DataFrame({'DATE':review_date_all,'STAR':review_star_all,'TITLE':review_title_all, 'USER':review_user_all,'BODY':review_body_all})
                columns_list = ['DATE','USER','STAR','TITLE','BODY']
                return(review_df[columns_list])
            else:
                body = res.text[6:]
                res_json = json.loads(body)
                bs = BeautifulSoup(res_json[0][2], 'html.parser')
                review_lists = bs.find_all('div',class_="single-review")
                for j in range(1,len(review_lists)):
                    review_date = review_lists[j].find('span',class_="review-date").text # review_date
                    review_star = int(re.findall(r'\d+', review_lists[0].find('div',class_="tiny-star star-rating-non-editable-container")['aria-label'])[1]) # Get Digits from Rates
                    review_title = review_lists[j].find('span',class_="review-title").text # title
                    review_body = review_lists[j].find('div',class_="review-body with-review-wrapper").text # review body
                    review_body = str.replace(review_body,review_title,"")
                    review_body = str.replace(review_body,"전체 리뷰","")
                    review_body = review_body.strip()
                    review_user = review_lists[j].find('span',class_="author-name").text.strip()  # get Reviewer
                    review_date_all.append(review_date)
                    review_star_all.append(review_star)
                    review_user_all.append(review_user)
                    review_title_all.append(review_title)
                    review_body_all.append(review_body)
        except:
            pass

In [ ]:
DF = GetReviews(url,id,token,10000)

 line : 20, code : 503
 line : 40, code : 503
 line : 60, code : 503
